In [1]:
# Step 1: Import Libraries and Load the Model
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

2025-06-27 19:35:30.944838: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-27 19:35:30.951819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751033130.960034   14671 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751033130.962607   14671 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751033130.969129   14671 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:

# Load the IMDB dataset word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [3]:
# Load the pre-trained model with ReLU activation
model = load_model('simple_rnn_imdb.h5')
model.summary()

I0000 00:00:1751033132.265358   14671 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9136 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
model.get_weights()

[array([[-0.01671417,  0.84433335,  0.02247376, ...,  1.5693456 ,
          0.27649668, -0.08896331],
        [-0.01210979,  0.0676417 ,  0.07450798, ...,  0.09044255,
         -0.03374785, -0.03541043],
        [ 0.04229767,  0.15869424,  0.11836717, ...,  0.07697936,
         -0.05990582,  0.15147002],
        ...,
        [-0.00425397, -0.07441056, -0.01275525, ...,  0.02481508,
          0.03513112, -0.03584582],
        [-0.0228104 , -0.0439289 , -0.05080882, ...,  0.13743754,
          0.00313594,  0.00724281],
        [-0.01297579, -0.03554784, -0.01053631, ...,  0.02957895,
          0.06342031, -0.00188209]], dtype=float32),
 array([[-0.05493634, -0.13811763, -0.05244634, ..., -0.04855314,
         -0.0836825 , -0.12663332],
        [-0.05041579, -0.14176482,  0.0113081 , ...,  0.08439913,
          0.05324774, -0.04171776],
        [ 0.08612095, -0.16921747,  0.03346435, ..., -0.13392226,
         -0.06675691, -0.12130336],
        ...,
        [ 0.04308471, -0.18161832, -0.1

In [5]:
# Step 2: Helper Functions
# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

In [6]:
### Prediction  function

def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)

    prediction=model.predict(preprocessed_input)

    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    
    return sentiment, prediction[0][0]



In [7]:
# Step 4: User Input and Prediction
# Example review for prediction
example_review = "This movie was fantastic! The acting was great and the plot was thrilling."

sentiment,score=predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

I0000 00:00:1751033132.847821   14814 service.cc:152] XLA service 0x7c0a6c004ab0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751033132.847836   14814 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 SUPER, Compute Capability 8.9
2025-06-27 19:35:32.858279: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
Review: This movie was fantastic! The acting was great and the plot was thrilling.
Sentiment: Negative
Prediction Score: 0.0845642238855362


I0000 00:00:1751033132.872804   14814 cuda_dnn.cc:529] Loaded cuDNN version 91002
I0000 00:00:1751033132.958714   14814 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
